In [1]:
#Enable API here: https://console.cloud.google.com/apis/api/earthengine.googleapis.com
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-hungweipan2-1')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

class FirePredictionModel:
    def __init__(self, roi_name, country_name):
        self.roi = ee.FeatureCollection("FAO/GAUL/2015/level1") \
                    .filter(ee.Filter.eq('ADM1_NAME', roi_name)) \
                    .filter(ee.Filter.eq('ADM0_NAME', country_name))

    def process_monthly_data(self, year, month):
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')

        # Fire data (label)
        fire = ee.ImageCollection('MODIS/061/MOD14A1') \
            .filterBounds(self.roi) \
            .filterDate(start_date, end_date) \
            .max() \
            .select('FireMask') \
            .expression("(b('FireMask') == 7 || b('FireMask') == 8 || b('FireMask') == 9) ? 1 : 0") \
            .rename('FireOccurred') \
            .toFloat()

        # NDVI data
        ndvi = ee.ImageCollection('MODIS/061/MOD13A2') \
            .filterBounds(self.roi) \
            .filterDate(start_date, end_date) \
            .mean() \
            .select('NDVI') \
            .multiply(0.0001) \
            .toFloat()

        # Weather data
        weather = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
            .filterBounds(self.roi) \
            .filterDate(start_date, end_date) \
            .mean() \
            .select([
                'temperature_2m',
                'dewpoint_temperature_2m',
                'u_component_of_wind_10m',
                'v_component_of_wind_10m'
            ])
        wind_speed = weather.expression(
            'sqrt(u**2 + v**2)', {
                'u': weather.select('u_component_of_wind_10m'),
                'v': weather.select('v_component_of_wind_10m')
            }
        ).rename('wind_speed')

        relative_humidity = weather.expression(
            '100 * (6.11 * pow(10, (7.5 * dew) / (237.7 + dew))) / (6.11 * pow(10, (7.5 * temp) / (237.7 + temp)))', {
                'temp': weather.select('temperature_2m').subtract(273.15),  # Convert to Celsius
                'dew': weather.select('dewpoint_temperature_2m').subtract(273.15)  # Convert to Celsius
            }


        # DEM data
        dem = ee.ImageCollection('NRCan/CDEM').mosaic().clip(self.roi).toFloat().rename('DEM')

        # Combine image
        combined = fire \
            .addBands(ndvi.rename('NDVI')) \
            .addBands(weather.select(['temperature_2m']).subtract(273.15).rename(['temp'])) \
            .addBands(relative_humidity) \
            .addBands(wind_speed) \
            .addBands(dem)
        return combined

    def collect_training_data(self, years, months, scale=1000):
        training_data = []
        for year in years:
            for month in months:
                image = self.process_monthly_data(year, month)
                samples = image.sample(
                    region=self.roi.geometry(),
                    scale=scale,
                    geometries=False
                )
                training_data.append(samples)
        return ee.FeatureCollection(training_data).flatten()


    def export_training_data(self, training_data, description, file_format='CSV'):
        task = ee.batch.Export.table.toDrive(
            collection=training_data,
            description=description,
            fileFormat=file_format
        )
        task.start()

# Initialize the model
model = FirePredictionModel(roi_name='Alberta', country_name='Canada')

# Define years and months
years = list(range(2023, 2024))
months = list(range(8, 9))

# Collect training data
training_data = model.collect_training_data(years, months)


# Export training data to Google Drive
model.export_training_data(training_data/, description='TrainingData2023_8')

print("Random Forest model training completed, and the training data export has started.")
